# Unit 6: Model-based Collaborative Filtering for **Ranking** Prediction

However, we still do Collaborative Filtering and Matrix Factorization in this unit, we do something fundamentally different: we change from rating prediction to **ranking prediction**.

We achieve this by changing the optimization criterion. Instead of minimizing the deviation between true and predicted ratings we push positive and negative user-item combinationa as much as possible apart. We transform explicit user feedback into implicit feedback. Implicit feedback refers to user interaction without the purpose to reflect preference or disregard and is much more common in pactice. Ranking prediction algorithms tackle to learn from implicit feedback data.

In addition, ranking-based algorithms yield a much more intuitive prediction result. Our goal is to present to the user a very limited amount of items in the correct ordering. Therefore, ordering is much more important than rating prediction. Ranking-based algorithms like BPR work pair-wise, i.e. for a user and two items they yield the correct order of both items for the user. Generalizing from this, we can impose an ordering on our item corpus and pick the top-$N$ to present to the user.

In [ ]:
from collections import OrderedDict
import itertools
from typing import Dict, List, Tuple

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
from recsys_training.data import Dataset
from recsys_training.evaluation import get_relevant_items

In [ ]:
ml100k_ratings_filepath = '../data/raw/ml-100k/u.data'

## Load Data

Different to previous units, we work with implicit feedback data now. However, MovieLens is an explicit feedback dataset, we can argue that everything above the user mean ratings is positive and everything below is negative. Bayesian Personalized Ranking learns from implicit positive feedback data and randomly samples negative feedback data during training. Thus, we keep all ratings above a threhold of $4.0$ and remove all other ratings.

In [ ]:
data = Dataset(ml100k_ratings_filepath)
data.filter(min_rating=4.0)
data.rating_split(seed=42)

As we want to learn the user/item latent factors from rating data, we first randomly initialize them

In [ ]:
seed = 42
m = data.n_users
n = data.n_items
d = 10

In [ ]:
# Latent Factor initialization
random_state = np.random.RandomState(seed)
user_factors = (random_state.rand(m, d) - 0.5) / d
item_factors = (random_state.rand(n, d) - 0.5) / d
        
ratings = data.train_ratings.sample(frac=1, random_state=seed)

In [ ]:
# positive implicit feedback items
user_pos_items = {}
# corpus of all remaining items for every user
# Ask me about the "Non missing at random hypothesis" ;)
user_neg_items = {}

In [ ]:
grouped = ratings[['user', 'item']].groupby('user')
groups = grouped.groups.keys()
for user in data.users:
    pos_items = []
    if user in groups:
        pos_items = grouped.get_group(user).item.values
    neg_items = np.setdiff1d(data.items, pos_items)
    user_pos_items[user] = pos_items
    user_neg_items[user] = neg_items

## Training

Yes, there is some math involved:

\begin{equation*}
\hat{x}_{uij} = \hat{x}_{ui} - \hat{x}_{uj} \\
x_{ui} = \sum_{f=1}^{d} w_{uf} \cdot h_{if}, i \in I_u^+ \\
x_{uj} = \sum_{f=1}^{d} w_{uf} \cdot h_{jf}, j \in I_u^- \\
\end{equation*}

\begin{equation*}
\text{BPR-Opt} := \sum_{(u,i,j) \in D_S} ln(\hat{x}_{uijj}) - \lambda_{\Theta} \cdot ||\Theta||^2
\end{equation*}

\begin{equation*}
\frac{\partial \text{BPR-Opt}}{\partial \Theta} = \frac{-e^{-\hat{x}_{uij}}}{1+e^{-\hat{x}_{uij}}} \cdot \frac{\partial \hat{x}_{uij}}{\partial \Theta} - \lambda_{\Theta} \cdot \Theta
\end{equation*}

\begin{equation*}
\frac{\partial x_{uij}}{\partial \Theta} =
\begin{cases}
(h_{if}-h_{jf}) & \text{for } \Theta = w_{uf} \\
w_{uf} & \text{for } \Theta = h_{if} \\
-w_{uf} & \text{for } \Theta = h_{jf}
\end{cases}
\end{equation*}

Let's talk about regularization!

In [ ]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

In [ ]:
def negative_sampling(user: int, user_neg_items) -> int:
    """
    Return the item ids for negative samples
    """
    # TODO: Allow for popularity-based pdf for choosing negative item

    return np.random.choice(user_neg_items[user])

**Task:** Adapt the `compute_gradients` method from the unit before to realize stochastic gradient descent (SGD) for Bayesian Personalized Ranking.

In [ ]:
def compute_gradients(user_embed: np.array,
                      pos_item_embed: np.array,
                      neg_item_embed: np.array,
                      l2_decay: Dict[str, float]) -> Tuple[np.array, np.array, np.array]:
    
    pass

    return user_grad, pos_item_grad, neg_item_grad

In [ ]:
def print_update(epoch: int, samples: np.array) -> float:
    # take the 1000 most recent ratings and compute the mean ranking loss
    users = samples[:, 0]
    pos_items = samples[:, 1]
    neg_items = np.array([negative_sampling(user, user_neg_items)
                          for user in users])

    user_embeds = user_factors[users - 1]
    pos_item_embeds = item_factors[pos_items - 1]
    neg_item_embeds = item_factors[neg_items - 1]

    pos_preds = np.sum(user_embeds * pos_item_embeds, axis=1)
    neg_preds = np.sum(user_embeds * neg_item_embeds, axis=1)
    preds = pos_preds - neg_preds

    loss = -np.log(sigmoid(preds)).mean()
    print(f"Epoch {epoch+1:02d}: Mean Ranking Loss: {loss:.4f}")
    
    return loss

Instead of minibatch gradient descent we do stochastic gradient descent here. It just shrinks the batch size down to 1 instance.

In [ ]:
epochs = 30
learning_rate = 0.05
l2_decay = {'user': 0.0, 'pos': 0.0, 'neg': 0.0}
verbose = True

In [ ]:
ratings_arr = ratings[['user', 'item']].values
n_ratings = len(ratings_arr)
loss_trace = []

for epoch in range(epochs):

    for _ in range(len(ratings)):
        random_index = np.random.randint(n_ratings)
        user, pos_item = tuple(ratings_arr[random_index])
        neg_item = negative_sampling(user, user_neg_items)

        # TODO: Align indices by making mapping user ids to compact and 0-indexed space
        # Deduct 1 as user ids are 1-indexed, but array is 0-indexed
        user_embed = user_factors[user - 1]
        pos_item_embed = item_factors[pos_item - 1]
        neg_item_embed = item_factors[neg_item - 1]

        user_grad, pos_item_grad, neg_item_grad = \
            compute_gradients(user_embed,
                              pos_item_embed,
                              neg_item_embed,
                              l2_decay)

        # update
        # TODO: Correct accordingly here and in Multi-Channel BPR Repo
        # update fails for multiple same users or items within a batch
        user_factors[user - 1] -= learning_rate * user_grad
        item_factors[pos_item - 1] -= learning_rate * pos_item_grad
        item_factors[neg_item - 1] -= learning_rate * neg_item_grad

    if verbose:
        samples = ratings_arr[-1000:]
        loss = print_update(epoch, samples)
        loss_trace.append(loss)

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(range(epochs), loss_trace, 'b--')
plt.grid(True)
plt.show()

### Using the model for Recommendations

We have now created a model to describe users and items in terms of latent vectors. But this time we fitted them to get the rankings correctly. So for obtaining recommendations we simply multiply user-item latent vectors we are interested in and achieve an estimate that can be used to order items for a given user. This time it is not a rating prediction, but still a prediction.

For that, we can reuse the `get_prediction` method from previous units.

Thus, before writing the `get_recommendations` again we first implement `get_prediction`.

In [ ]:
def get_prediction(user: int, items: np.array = None, remove_known_pos: bool = True) -> Dict[int, Dict[str, float]]:
    if items is None:
        if remove_known_pos:
            # Predict from unobserved items
            # We simplified this compared to the unit before
            items = user_neg_items[user]
        else:
            items = np.array(data.items)
    if type(items) == np.int64:
        items = np.array([items])
    
    user_embed = user_factors[user - 1].reshape(1, -1)
    item_embeds = item_factors[items - 1].reshape(len(items), -1)

    # use array-broadcasting
    preds = np.sum(user_embed * item_embeds, axis=1)
    sorting = np.argsort(preds)[::-1]
    preds = {item: {'pred': pred} for item, pred in
             zip(items[sorting], preds[sorting])}

    return preds

In [ ]:
item_predictions = get_prediction(1)

In [ ]:
list(item_predictions.items())[:20]

In [ ]:
def get_recommendations(user: int, N: int, remove_known_pos: bool = False) -> List[Tuple[int, Dict[str, float]]]:
    predictions = get_prediction(user, remove_known_pos=remove_known_pos)
    recommendations = []
    for item, pred in predictions.items():
        add_item = (item, pred)
        recommendations.append(add_item)
        if len(recommendations) == N:
            break

    return recommendations

In [ ]:
recommendations = get_recommendations(1, 10)

In [ ]:
recommendations

## Evaluation

In [ ]:
N = 10

In [ ]:
relevant_items = get_relevant_items(data.test_ratings)

In [ ]:
users = relevant_items.keys()
prec_at_N = dict.fromkeys(data.users)

for user in users:
    recommendations = get_recommendations(user, N, remove_known_pos=True)
    recommendations = [val[0] for val in recommendations]
    hits = np.intersect1d(recommendations,
                          relevant_items[user])
    prec_at_N[user] = len(hits)/N

In [ ]:
recommendations

In [ ]:
np.mean([val for val in prec_at_N.values() if val is not None])

**BAM!**